# 単位円盤上のPoisson方程式

MathWorkの以下の[問題](https://jp.mathworks.com/help/pde/ug/solve-poissons-equation-on-a-unit-disk.html)が面白そうだったのでGetFEM++で解いてみます。円盤の半径は0.5で、円盤内ではPoisson方程式が成り立っているとします。円の境界にはDirichlet条件が設けられています。
![Geometory](https://jp.mathworks.com/help/examples/pde/win64/pdedemo1_01.png)
MathWorksより引用

In [2]:
import getfem as gf

h = 0.1

In [3]:
mo = gf.MesherObject('ball', [0.5, 0.5], 0.5)

In [4]:
mesh = gf.Mesh('generate', mo, h, 2)

In [5]:
mesh.export_to_vtk('mesh.vtk', 'ascii')